In [27]:
import numpy as np
import pandas as pd
from scipy.stats import t, norm
from scipy.optimize import minimize

file = 'problem1.csv'
data = pd.read_csv(file)
returns = data['x'].values

Normal Distribution with an Exponentially Weighted Variance

In [38]:
alpha = 0.95
lambda_value = 0.97


In [52]:
def exponentially_weighted_variance(returns, lambda_value):
    weights = np.array([(1 - lambda_value) * (lambda_value ** i) for i in range(len(returns))])[::-1]
    weighted_mean = np.mean(returns)
    weighted_variance = np.dot(weights, (returns - weighted_mean) ** 2)
    return weighted_mean, np.sqrt(weighted_variance)

mean_return, std_dev = exponentially_weighted_variance(returns, lambda_value)

VaR = norm.ppf(1 - alpha) * std_dev

f_alpha = norm.pdf(norm.ppf(1 - alpha))
ES = -mean_return + std_dev * f_alpha / (1-alpha)

print("Value at Risk (VaR):", VaR)
print("Expected Shortfall (ES):", ES)

Value at Risk (VaR): -0.09028950269062073
Expected Shortfall (ES): 0.11410650846697049


MLE fitted T distribution

In [11]:
def neg_log_likelihood(params, data):
    nu, mu, sigma = params
    return -np.sum(t.logpdf(data, df=nu, loc=mu, scale=sigma))

initial_params = [10, np.mean(returns), np.std(returns)]

bounds = [(2.1, None), (None, None), (0.001, None)]

result = minimize(neg_log_likelihood, initial_params, args=(returns,), bounds=bounds)

nu_mle, mu_mle, sigma_mle = result.x

VaR_t = t.ppf(1 - alpha, df=nu_mle, loc=mu_mle, scale=sigma_mle)

def expected_shortfall_t(alpha, nu, mu, sigma):
    x_alpha = t.ppf(1 - alpha, df=nu)
    pdf_alpha = t.pdf(x_alpha, df=nu)
    cdf_alpha = 1 - alpha
    ES = mu - sigma * (pdf_alpha * (nu + x_alpha**2) / (nu - 1)) / cdf_alpha
    return ES

ES_t = expected_shortfall_t(alpha, nu_mle, mu_mle, sigma_mle)

print("Value at Risk (VaR) using T-distribution:", VaR_t)
print("Expected Shortfall (ES) using T-distribution:", ES_t)

Value at Risk (VaR) using T-distribution: -0.0764758759203622
Expected Shortfall (ES) using T-distribution: -0.11321759703671269


Historic Simulation

In [12]:
def historical_var(returns, alpha):
    sorted_returns = np.sort(returns)
    var_index = int((1 - alpha) * len(sorted_returns))
    VaR_historical = sorted_returns[var_index]
    return VaR_historical

def historical_es(returns, alpha):
    VaR_historical = historical_var(returns, alpha)
    ES_historical = returns[returns <= VaR_historical].mean()
    return ES_historical

VaR_hist = historical_var(returns, alpha)
ES_hist = historical_es(returns, alpha)

print("Value at Risk (VaR) using Historical Simulation:", VaR_hist)
print("Expected Shortfall (ES) using Historical Simulation:", ES_hist)

Value at Risk (VaR) using Historical Simulation: -0.075861511162783
Expected Shortfall (ES) using Historical Simulation: -0.11520303685782037
